In [1]:
!pip uninstall -y torchvision torchaudio pytorch-lightning lightning torch-tensorrt

!pip install torchvision
!pip install torchaudio

# Install compatible PyTorch Lightning
!pip install pytorch-lightning>=2.2.0

Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
Found existing installation: torchaudio 2.5.1+cu124
Uninstalling torchaudio-2.5.1+cu124:
  Successfully uninstalled torchaudio-2.5.1+cu124
Found existing installation: pytorch-lightning 2.5.1
Uninstalling pytorch-lightning-2.5.1:
  Successfully uninstalled pytorch-lightning-2.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 111.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install -q basicsr realesrgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 17.0 MB/s eta 0:00:00


In [4]:
# Run this BEFORE importing basicsr
import importlib
import sys
import types
import torch
import torchvision

# Define the missing function directly
def rgb_to_grayscale_patch(tensor, num_output_channels=1):
    
    if tensor.shape[-3] != 3:
        raise ValueError("Input tensor must have 3 channels (RGB)")
        
    # Use the existing implementation from functional
    from torchvision.transforms import functional as F
    return F.rgb_to_grayscale(tensor, num_output_channels)

# Create a module-like object
functional_tensor = types.ModuleType('torchvision.transforms.functional_tensor')
functional_tensor.rgb_to_grayscale = rgb_to_grayscale_patch

# Add the module to sys.modules so it can be imported
sys.modules['torchvision.transforms.functional_tensor'] = functional_tensor

# Also add it to torchvision.transforms
torchvision.transforms.functional_tensor = functional_tensor

print("Created and registered torchvision.transforms.functional_tensor with rgb_to_grayscale function")

Created and registered torchvision.transforms.functional_tensor with rgb_to_grayscale function


In [5]:
import torch
print(hasattr(torch._dynamo, "mark_static_address"))

True


In [6]:
import torch

# Check PyTorch version
print(f"PyTorch Version: {torch.__version__}")

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available, running on CPU.")


PyTorch Version: 2.7.0+cu126
CUDA is available! GPU: Tesla T4


In [7]:
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from realesrgan import RealESRGANer

In [11]:
import os
import cv2
import gc
import torch
import numpy as np
from tqdm import tqdm
from datetime import datetime

# RealESRGAN imports
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from realesrgan import RealESRGANer

class VideoEnhancer:
    def __init__(self, model_name='RealESRGAN_x4plus', model_path=None, scale=4,
                 denoise_strength=0.5, tile_size=512, half_precision=None, 
                 use_cuda=True, output_format='mp4v'):
        """
        Initialize the VideoEnhancer with specified parameters.
        
        Args:
            model_name (str): Name of the model to use.
            model_path (str): Path to the model weights.
            scale (int): Scale factor for upscaling.
            denoise_strength (float): Strength of denoising.
            tile_size (int): Tile size for processing large frames.
            half_precision (bool): Whether to use half precision (FP16).
            use_cuda (bool): Whether to use CUDA if available.
            output_format (str): Format code for the output video.
        """
        # Set CUDA availability
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.device = torch.device('cuda' if self.use_cuda else 'cpu')
        
        # Store parameters
        self.scale = scale
        self.model_name = model_name
        self.denoise_strength = denoise_strength
        self.upsampler = None
        self.output_format = output_format
        
        # Set half precision based on CUDA availability if not specified
        if half_precision is None:
            half_precision = self.use_cuda

        # Adjust tile size based on available GPU memory
        self.tile_size = tile_size
        if self.use_cuda:
            try:
                gpu_mem = torch.cuda.get_device_properties(0).total_memory / (1024**3)
                if gpu_mem < 4 and tile_size > 256:  # Less than 4GB VRAM
                    self.tile_size = 256
                elif gpu_mem < 8 and tile_size > 512:  # Less than 8GB VRAM
                    self.tile_size = 512
                print(f"Detected {gpu_mem:.2f}GB GPU memory, using tile size: {self.tile_size}")
            except Exception as e:
                print(f"Error detecting GPU memory: {e}")
                print(f"Using default tile size: {self.tile_size}")

        # Determine the model path
        if model_path and os.path.exists(model_path) and os.path.getsize(model_path) > 1000:
            print(f"Using provided model path: {model_path}")
        else:
            model_path = os.path.join('weights', f"{model_name}.pth")
            
            if not os.path.exists(model_path) or os.path.getsize(model_path) < 1000:
                # Create weights directory if it doesn't exist
                os.makedirs('weights', exist_ok=True)
                
                # Download model weights
                if model_name == 'RealESRGAN_x4plus':
                    model_url = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth'
                    load_file_from_url(model_url, model_dir='weights')
                elif model_name == 'RealESRGAN_x4plus_anime_6B':
                    model_url = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth'
                    load_file_from_url(model_url, model_dir='weights')
                else:
                    raise FileNotFoundError(f"Model file not found at {model_path} and no download URL for {model_name}")

        print(f"Using {model_name} model from: {model_path} with {scale}x upscaling")
        
        try:
            # Initialize RealESRGAN model
            if model_name == 'RealESRGAN_x4plus':
                model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=self.scale)
            elif model_name == 'RealESRGAN_x4plus_anime_6B':
                model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=self.scale)
            else:
                raise ValueError(f"Unsupported model name: {model_name}")
            
            # Create the upsampler
            self.upsampler = RealESRGANer(
                scale=self.scale,
                model_path=model_path,
                model=model,
                tile=self.tile_size,
                tile_pad=32,
                pre_pad=0,
                half=half_precision,
                device=self.device
            )
            
        except Exception as e:
            raise RuntimeError(f"Failed to initialize model {model_name}: {e}")
        
        self.model_attributes = {
            'name': model_name,
            'path': model_path,
            'scale': scale,
            'tile_size': self.tile_size,
            'half_precision': half_precision,
            'device': str(self.device)
        }
        
        print(f"VideoEnhancer initialized with {model_name} on {self.device}")
    
    def enhance_frame(self, frame):
        """
        Enhance a single video frame.
        
        Args:
            frame: Input frame (OpenCV format, BGR)
            
        Returns:
            Enhanced frame in the same format
        """
        try:
            h, w = frame.shape[:2]
            max_dimension = 1280  # Maximum dimension to process at once
            scale_factor = 1

            # Resize large frames to avoid memory issues
            if max(h, w) > max_dimension:
                scale_factor = max_dimension / max(h, w)
                new_h, new_w = int(h * scale_factor), int(w * scale_factor)
                frame = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)
                print(f"Resized frame from {h}x{w} to {new_h}x{new_w} for processing")

            # Convert BGR to RGB for processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Clear CUDA cache before processing
            if self.use_cuda:
                torch.cuda.empty_cache()

            # Enhance the frame
            output, _ = self.upsampler.enhance(frame_rgb, outscale=self.scale)

            # Convert back to BGR
            output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)

            # Clear CUDA cache after processing
            if self.use_cuda:
                torch.cuda.empty_cache()

            return output

        except RuntimeError as e:
            if 'out of memory' in str(e).lower():
                print("WARNING: GPU out of memory. Trying with smaller frame...")
                # If we hit OOM, resize frame more aggressively and try again
                h, w = frame.shape[:2]
                new_h, new_w = int(h * 0.5), int(w * 0.5)
                small_frame = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)

                if self.use_cuda:
                    torch.cuda.empty_cache()

                enhanced_small = self.enhance_frame(small_frame)  # Recursive call with smaller frame
                return cv2.resize(enhanced_small, (w*self.scale, h*self.scale), interpolation=cv2.INTER_LANCZOS4)
            else:
                print(f"Error enhancing frame: {e}")
                # Return upscaled original frame if enhancement fails
                return cv2.resize(frame, (int(frame.shape[1]*self.scale), int(frame.shape[0]*self.scale)), 
                                interpolation=cv2.INTER_LANCZOS4)
        except Exception as e:
            print(f"Error enhancing frame: {e}")
            # Return upscaled original frame if enhancement fails
            return cv2.resize(frame, (int(frame.shape[1]*self.scale), int(frame.shape[0]*self.scale)), 
                            interpolation=cv2.INTER_LANCZOS4)
    
    def enhance_video(self, input_path, output_path=None, start_time=0, duration=None, 
                     sample_rate=1, resize_factor=1.0, show_progress=True, audio=True):
        """
        Enhance a video file.
        
        Args:
            input_path (str): Path to the input video.
            output_path (str): Path for the enhanced output video. If None, a default name will be used.
            start_time (float): Start time in seconds to begin processing.
            duration (float): Duration in seconds to process. If None, process until the end.
            sample_rate (int): Process 1 out of every 'sample_rate' frames.
            resize_factor (float): Factor to resize the original frames before enhancing.
            show_progress (bool): Whether to show a progress bar.
            audio (bool): Whether to copy audio from the original video.
            
        Returns:
            str: Path to the enhanced video.
        """
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Input video not found: {input_path}")
        
        # Generate default output path if not provided
        if output_path is None:
            basename = os.path.basename(input_path)
            name, ext = os.path.splitext(basename)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = f"{name}_enhanced_{timestamp}{ext}"
        
        # Create output directory if it doesn't exist
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)
        
        # Open the input video
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise RuntimeError(f"Could not open video: {input_path}")
        
        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Calculate frame positions for start_time and duration
        start_frame = int(start_time * fps)
        if duration is not None:
            end_frame = start_frame + int(duration * fps)
        else:
            end_frame = frame_count
        
        # Calculate the total frames to process considering sample_rate
        total_frames = (end_frame - start_frame) // sample_rate
        
        # Print video information
        print(f"Video Info: {width}x{height}, {fps} fps, {frame_count} frames")
        print(f"Processing {total_frames} frames from frame {start_frame} to {end_frame} (sample rate: {sample_rate})")
        
        # Calculate new dimensions based on resize_factor and scale
        if resize_factor != 1.0:
            width = int(width * resize_factor)
            height = int(height * resize_factor)
            print(f"Resizing input frames to: {width}x{height}")
        
        # Calculate output dimensions based on scale
        out_width = int(width * self.scale)
        out_height = int(height * self.scale)
        print(f"Output dimensions: {out_width}x{out_height}")
        
        # Find the appropriate video codec
        if self.output_format == 'mp4v':
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Default MP4 codec
        elif self.output_format == 'avc1':
            fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264 codec
        elif self.output_format == 'H264':
            fourcc = cv2.VideoWriter_fourcc(*'H264')  # Another variant of H.264
        elif self.output_format == 'XVID':
            fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Xvid codec
        else:
            # Fallback to MJPG which is widely supported
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            output_path = os.path.splitext(output_path)[0] + '.avi'
            print(f"Using MJPG codec with .avi container. Output: {output_path}")
        
        # Create a video writer
        out = cv2.VideoWriter(output_path, fourcc, fps, (out_width, out_height))
        if not out.isOpened():
            raise RuntimeError(f"Could not create output video writer: {output_path}")
        
        # Set the starting position
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        # Process the video frame by frame
        processed_frames = 0
        current_frame = start_frame
        
        try:
            # Create progress bar if requested
            pbar = tqdm(total=total_frames, disable=not show_progress)
            
            while current_frame < end_frame:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process 1 out of every sample_rate frames
                if (current_frame - start_frame) % sample_rate == 0:
                    # Resize input frame if needed
                    if resize_factor != 1.0:
                        frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
                    
                    # Enhance the frame
                    enhanced_frame = self.enhance_frame(frame)
                    
                    # Write the enhanced frame
                    out.write(enhanced_frame)
                    
                    # Update progress
                    processed_frames += 1
                    pbar.update(1)
                    
                    # Free memory
                    del enhanced_frame
                    if self.use_cuda:
                        torch.cuda.empty_cache()
                
                current_frame += 1
                
                # Keyboard interrupt check (press 'q' to stop)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    print("\nProcessing interrupted by user")
                    break
            
            pbar.close()
        
        except Exception as e:
            print(f"Error during video processing: {e}")
            import traceback
            traceback.print_exc()
        finally:
            # Release resources
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            
            # Force garbage collection
            gc.collect()
            if self.use_cuda:
                torch.cuda.empty_cache()
        
        print(f"Processed {processed_frames} frames. Enhanced video saved to: {output_path}")
        
        # Add audio from the original video if requested
        if audio:
            try:
                temp_output = output_path + ".temp" + os.path.splitext(output_path)[1]
                os.rename(output_path, temp_output)
                
                # Use FFmpeg to add audio from the original video
                import subprocess
                ffmpeg_cmd = [
                    'ffmpeg', '-i', temp_output, '-i', input_path, 
                    '-c:v', 'copy', '-map', '0:v:0', '-map', '1:a:0?', 
                    '-shortest', output_path, '-y'
                ]
                
                print("Adding audio from original video...")
                subprocess.run(ffmpeg_cmd, check=True, capture_output=True)
                
                # Remove the temporary file
                os.remove(temp_output)
                print("Audio added successfully")
            except Exception as e:
                print(f"Warning: Could not add audio: {e}")
                # If adding audio fails, rename the temp file back
                if os.path.exists(temp_output):
                    os.rename(temp_output, output_path)
        
        return output_path
    
    def get_model_info(self):
        """Return information about the currently loaded model"""
        return self.model_attributes


def enhance_video_file(input_path, output_path=None, model_name='RealESRGAN_x4plus', model_path=None, 
                     scale=2, tile_size=512, use_cuda=True, start_time=0, duration=None, 
                     sample_rate=1, resize_factor=1.0, copy_audio=True, output_format='mp4v'):
    """
    Simple function to enhance a video file with default parameters.
    
    Args:
        input_path (str): Path to the input video file
        output_path (str): Path for the enhanced output video. If None, a default name will be used.
        model_name (str): Model to use ('RealESRGAN_x4plus' or 'RealESRGAN_x4plus_anime_6B')
        model_path (str): Optional path to model weights
        scale (int): Upscaling factor (2, 3, or 4)
        tile_size (int): Tile size for processing large frames
        use_cuda (bool): Whether to use CUDA if available
        start_time (float): Start time in seconds
        duration (float): Duration to process in seconds (None for full video)
        sample_rate (int): Process 1 out of every N frames
        resize_factor (float): Resize input frames before enhancement
        copy_audio (bool): Whether to copy audio from original video
        output_format (str): Output video codec format ('mp4v', 'avc1', 'H264', 'XVID')
    
    Returns:
        str: Path to the enhanced video
    """
    # Initialize the enhancer
    enhancer = VideoEnhancer(
        model_name=model_name,
        model_path=model_path,
        scale=scale,
        tile_size=tile_size,
        use_cuda=use_cuda,
        output_format=output_format
    )
    
    # Enhance the video
    output_path = enhancer.enhance_video(
        input_path=input_path,
        output_path=output_path,
        start_time=start_time,
        duration=duration,
        sample_rate=sample_rate,
        resize_factor=resize_factor,
        audio=copy_audio
    )
    
    print(f"Enhancement complete! Output saved to: {output_path}")
    return output_path


def main():
    """Example usage of the video enhancer"""
    # Example: Enhance a video with default settings
    # input_video = "/kaggle/input/videodemo/enhanced_demo1.mp4"  # Replace with your video path
    # enhance_video_file(input_video)
    
    # Example: Enhance with custom settings
    enhance_video_file(
        input_path="/kaggle/input/demo1enhance/demo.mp4",
        output_path="enhanced_output.mp4",
        model_name="RealESRGAN_x4plus",
        scale=4,
        sample_rate=2,  # Process every other frame for speed
        resize_factor=0.5  # Resize to half size before enhancing
    )


if __name__ == "__main__":
    main()

Detected 14.74GB GPU memory, using tile size: 512
Using RealESRGAN_x4plus model from: weights/RealESRGAN_x4plus.pth with 4x upscaling
VideoEnhancer initialized with RealESRGAN_x4plus on cuda
Video Info: 1920x1080, 18.0 fps, 199 frames
Processing 99 frames from frame 0 to 199 (sample rate: 2)
Resizing input frames to: 960x540
Output dimensions: 3840x2160



  0%|          | 0/99 [00:00<?, ?it/s]

	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4



  1%|          | 1/99 [00:04<06:32,  4.00s/it]

	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4


...

	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4



100%|██████████| 99/99 [07:01<00:00,  4.43s/it]

	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4



100it [07:05,  4.26s/it]                       


Processed 100 frames. Enhanced video saved to: enhanced_output.mp4
Adding audio from original video...
Audio added successfully
Enhancement complete! Output saved to: enhanced_output.mp4
